<a href="https://colab.research.google.com/github/JSJeong-me/Forecast/blob/main/GRPO/03-grpo_midprice_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GRPO 기반 실시간 중간가 예측 파이프라인 (Python 3.11 / CUDA 12.1 Colab)

이 노트북은 **Python 3.11** Colab GPU 환경에서 추가 패키지 충돌 없이 실행되도록
설계되었습니다. critic 없이 배치 평균 보상을 베이스라인으로 사용하는
**경량 GRPO(Group Relative Policy Optimization)** 구현을 통해  
하루치 **깊이‑1 호가** 스냅숏 CSV(`stock-sec-20250703.csv`)로 학습한 뒤  
`H = 10` 틱 후 중간가를 실시간 예측하는 과정을 단계별로 보여줍니다.

> ⚠️ _사용 전_: `CSV` 경로를 본인 Drive 위치에 맞게 수정하세요.


In [ ]:
# %%bash
# Colab 런타임(파이썬 3.11)은 CUDA 12.1 지원 PyTorch 2.x wheel 사용
!pip install -q --upgrade pip
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install -q gymnasium numpy pandas


In [5]:
# prompt: google drive mount

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import pandas as pd, numpy as np, torch, torch.nn as nn, torch.optim as optim
import gymnasium as gym

CSV = '/content/drive/MyDrive/Working-2025/stock/stock-sec-20250703.csv'  # <-- 수정
df   = pd.read_csv(CSV, parse_dates=['timestamp']).sort_values('timestamp')

# 한글 컬럼 → 영문
col_map = {'매도호가01':'ask_p1', '매수호가01':'bid_p1',
           '매도호가잔량01':'ask_s1', '매수호가잔량01':'bid_s1',
           '총매도호가 잔량':'tot_ask_size', '총매수호가 잔량':'tot_bid_size'}
df.rename(columns=col_map, inplace=True)

# 파생 특징
df['mid']    = (df['ask_p1'] + df['bid_p1']) / 2
df['spread'] =  df['ask_p1'] - df['bid_p1']
d_bid        = df['bid_s1'].diff().fillna(0)
d_ask        = df['ask_s1'].diff().fillna(0)
df['ofi']    = np.where(df['bid_p1'].diff()>=0,  d_bid,-d_bid) +                np.where(df['ask_p1'].diff()<=0, -d_ask, d_ask)
df['imb']    = (df['tot_bid_size']-df['tot_ask_size']) / (df['tot_bid_size']+df['tot_ask_size']+1e-9)

print("데이터 준비 완료 :", df.shape)

데이터 준비 완료 : (9081, 49)


In [8]:
class LOBMidEnv(gym.Env):
    metadata = {"render_modes": []}
    def __init__(self, frame=20, horizon=10):
        super().__init__()
        self.df, self.f, self.h = df.reset_index(drop=True), frame, horizon
        self.feats = ['ask_p1','bid_p1','ask_s1','bid_s1','spread','ofi','imb']
        low  = -np.inf*np.ones((frame, len(self.feats)), dtype=np.float32)
        high =  np.inf*np.ones_like(low)
        self.observation_space = gym.spaces.Box(low, high)
        self.action_space      = gym.spaces.Box(-np.inf, np.inf, (1,), dtype=np.float32)
    def reset(self, *, seed=None, options=None):
        self.t = self.f
        return self._obs(), {}
    def step(self, action):
        pred  = action[0]
        true  = self.df.loc[self.t + self.h, 'mid']
        reward = -abs(pred - true)       # 음의 절대오차 → 보상
        self.t += 1
        done   = self.t + self.h >= len(self.df) - 1
        return self._obs(), reward, done, False, {}
    def _obs(self):
        return self.df[self.feats].iloc[self.t-self.f:self.t].values.astype(np.float32)


In [9]:
import torch.nn as nn
import torch.optim as optim

class Policy(nn.Module):
    def __init__(self, n_feat):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(n_feat, 32, 3, padding=1), nn.ReLU(),
            nn.Conv1d(32, 64, 3, padding=1), nn.ReLU(),
            nn.AdaptiveAvgPool1d(1), nn.Flatten(),
            nn.Linear(64, 32), nn.ReLU(),
            nn.Linear(32, 1)
        )
    def forward(self, x):               # x: (B, frame, n_feat)
        return self.net(x.transpose(1, 2))

class GRPOAgent:
    def __init__(self, env, device='cuda'):
        self.env    = env
        self.device = device
        self.pi     = Policy(len(env.feats)).to(device)
        self.opt    = optim.Adam(self.pi.parameters(), lr=3e-4)
        self.clip   = 0.2
    def collect(self, batch=4096):
        obs, acts, rews, logps = [], [], [], []
        o, _ = self.env.reset()
        while len(rews) < batch:
            with torch.no_grad():
                a = self.pi(torch.tensor(o).unsqueeze(0).to(self.device))[0,0].cpu().item()
            n_o, r, d, _, _ = self.env.step([a])
            obs.append(o); acts.append([a]); rews.append(r)
            o = n_o
            if d:
                o, _ = self.env.reset()
        O = torch.tensor(obs, dtype=torch.float32).to(self.device)
        A = torch.tensor(acts, dtype=torch.float32).to(self.device)
        R = torch.tensor(rews, dtype=torch.float32).to(self.device)
        logp = -((A - self.pi(O).detach())**2).sum(dim=1)
        return O, A, R, logp
    def update(self, O, A, R, old_logp, epochs=4, group=32):
        for _ in range(epochs):
            idx = torch.randperm(len(R))
            for i in range(0, len(R), group):
                j = idx[i:i+group]
                o, a, r, ol = O[j], A[j], R[j], old_logp[j]
                logp  = -((a - self.pi(o))**2).sum(dim=1)
                adv   = r - r.mean()                 # 그룹 평균 베이스라인
                ratio = torch.exp(logp - ol)
                pg    = torch.minimum(ratio*adv,
                                      torch.clamp(ratio, 1-self.clip, 1+self.clip)*adv)
                loss  = -pg.mean()
                self.opt.zero_grad(); loss.backward(); self.opt.step()

In [10]:
env   = LOBMidEnv()
agent = GRPOAgent(env)

for epoch in range(30):
    O, A, R, logp = agent.collect(batch=2048)
    agent.update(O, A, R, logp)
    print(f"Epoch {epoch:02d} | 평균 보상 {R.mean().item():.4f}")


/tmp/ipython-input-9-2796751951.py:35: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  O = torch.tensor(obs, dtype=torch.float32).to(self.device)


Epoch 00 | 평균 보상 -61521.8984
Epoch 01 | 평균 보상 -62189.1094
Epoch 02 | 평균 보상 -68118.6797
Epoch 03 | 평균 보상 -72666.9219
Epoch 04 | 평균 보상 -71406.8828
Epoch 05 | 평균 보상 -73578.1094
Epoch 06 | 평균 보상 -68991.5234
Epoch 07 | 평균 보상 -67870.8750
Epoch 08 | 평균 보상 -67343.7422
Epoch 09 | 평균 보상 -69531.3672
Epoch 10 | 평균 보상 -66601.5000
Epoch 11 | 평균 보상 -64695.3906
Epoch 12 | 평균 보상 -71665.0078
Epoch 13 | 평균 보상 -71362.7500
Epoch 14 | 평균 보상 -71729.7969
Epoch 15 | 평균 보상 -70088.9844
Epoch 16 | 평균 보상 -71230.7891
Epoch 17 | 평균 보상 -68802.9062
Epoch 18 | 평균 보상 -65739.0312
Epoch 19 | 평균 보상 -66082.2188
Epoch 20 | 평균 보상 -65596.0625
Epoch 21 | 평균 보상 -65760.7656
Epoch 22 | 평균 보상 -65980.9375
Epoch 23 | 평균 보상 -73687.0625
Epoch 24 | 평균 보상 -74906.4531
Epoch 25 | 평균 보상 -78595.5000
Epoch 26 | 평균 보상 -81046.8125
Epoch 27 | 평균 보상 -70890.6094
Epoch 28 | 평균 보상 -73480.7656
Epoch 29 | 평균 보상 -71228.7344


In [11]:
state, _ = env.reset()
with torch.no_grad():
    pred_mid = agent.pi(torch.tensor(state).unsqueeze(0).to(agent.device))[0,0].cpu().item()
print("실시간(+10틱) 예측 중간가 :", round(pred_mid, 2))


실시간(+10틱) 예측 중간가 : -7363.64
